In [2]:
import torch
import torch.nn as nn

import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [3]:
class MyNetwork(nn.Module):

    def __init__(self, input_dim=20, hidden_dim1=64, hidden_dim2=128, hidden_dim3=64, output_dim=1):
        super(MyNetwork, self).__init__()
        self.layer1 = nn.Linear(input_dim, hidden_dim1)
        self.layer2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.layer3 = nn.Linear(hidden_dim2, hidden_dim3)
        self.output_layer = nn.Linear(hidden_dim1 + hidden_dim3, output_dim)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # First layer
        out1 = self.relu(self.layer1(x))

        # Second layer
        out2 = self.relu(self.layer2(out1))

        # Third layer
        out3 = self.relu(self.layer3(out2))

        # Concatenate outputs of the 1st and 3rd layers
        concatenated_output = torch.cat((out1, out3), dim=1)

        # Output layer
        output = self.sigmoid(self.output_layer(concatenated_output))

        return output

In [4]:
# x = torch.randn(32, 20)  # Batch of 32 samples with 20 features each
# model = MyNetwork()
# output = model(x)
# print(output.shape)  # Should print torch.Size([32, 1])

In [5]:
def generate_synthetic_data(num_samples=1000, input_dim=20):
    # Generate random features
    X = torch.randn(num_samples, input_dim)

    # Generate random labels (0 or 1)
    y = torch.randint(0, 2, (num_samples, 1)).float()

    return X, y

In [6]:
def train_model(model, dataloader, criterion, optimizer, num_epochs=10, device='cpu'):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)


            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')

    print('Training complete.')


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [8]:

# Hyperparameters
input_dim = 20
hidden_dim1 = 64
hidden_dim2 = 128
hidden_dim3 = 64
output_dim = 1
batch_size = 32
learning_rate = 0.001
num_epochs = 10

# Initialize the model, criterion, and optimizer
model = MyNetwork(input_dim=input_dim, hidden_dim1=hidden_dim1, hidden_dim2=hidden_dim2, hidden_dim3=hidden_dim3, output_dim=output_dim).to(device)
criterion = nn.BCELoss()

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Generate synthetic data
X, y = generate_synthetic_data(num_samples=1000, input_dim=input_dim)

# Create DataLoader
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Train the model
train_model(model, dataloader, criterion, optimizer, num_epochs=num_epochs, device=device)

Epoch [1/10], Loss: 0.6934
Epoch [2/10], Loss: 0.6864
Epoch [3/10], Loss: 0.6807
Epoch [4/10], Loss: 0.6758
Epoch [5/10], Loss: 0.6637
Epoch [6/10], Loss: 0.6504
Epoch [7/10], Loss: 0.6312
Epoch [8/10], Loss: 0.6025
Epoch [9/10], Loss: 0.5697
Epoch [10/10], Loss: 0.5201
Training complete.


In [9]:
def evaluate_model(model, dataloader, device='cpu'):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # No need to calculate gradients during evaluation
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted = (outputs > 0.5).float()  # Apply threshold for binary classification
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Accuracy: {accuracy * 100:.2f}%')

def make_inference(model, input_data, device='cpu'):
    model.eval()  # Set the model to evaluation mode
    input_data = input_data.to(device)
    with torch.no_grad():  # No need to calculate gradients during inference
        output = model(input_data)
        predicted = (output > 0.5).float()  # Apply threshold for binary classification
    return predicted

# Example usage:

# Generate synthetic test data (similar to training data)
X_test, y_test = generate_synthetic_data(num_samples=200, input_dim=input_dim)

# Create DataLoader for test data
test_dataset = TensorDataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Evaluate the model on the test data
evaluate_model(model, test_dataloader, device=device)

# Example of making predictions on a new input
new_input = torch.randn(1, input_dim)  # Generate a random new input sample
predicted_label = make_inference(model, new_input, device=device)
print(f'Predicted label: {predicted_label.item()}')


Accuracy: 53.00%
Predicted label: 1.0
